In [10]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns
import numpy as np

from tqdm import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
from pathlib import Path


from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.examples.ubc_plotting_utils import *
from open_spiel.python.examples.ubc_sample_game_tree import sample_game_tree, flatten_trees, flatten_tree
from open_spiel.python.examples.ubc_cma import analyze_checkpoint
from open_spiel.python.pytorch.auction_convnet import AuctionConvNet, AuctionNet


from auctions.webutils import *

import bokeh
from bokeh.layouts import row, column
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource, ColorBar, LogColorMapper, LinearColorMapper
from bokeh.transform import linear_cmap, log_cmap
from bokeh.palettes import Magma256
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

output_notebook()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

In [27]:
observation_shape = (10, 3, 3, 3)
num_observations = 20
data = torch.rand(num_observations, *one_product_shape)

network = AuctionConvNet(observation_shape, [50, 50], add_skip_connections=False)

In [78]:

one_product_shape = (1, 1, 3)
data = torch.rand(20, *one_product_shape)
# print(data)
# print(data.shape)
expanded_data = data.expand(20, *observation_shape[1:])

In [76]:
$x + y$

SyntaxError: invalid syntax (787750645.py, line 1)

In [72]:
data.squeeze()

tensor([[0.9681, 0.1617, 0.0027],
        [0.0947, 0.6865, 0.0041],
        [0.1417, 0.8920, 0.0695],
        [0.3857, 0.0498, 0.2806],
        [0.5163, 0.1771, 0.7698],
        [0.1702, 0.2417, 0.7978],
        [0.3985, 0.9789, 0.3466],
        [0.8185, 0.9099, 0.7953],
        [0.1717, 0.5009, 0.0099],
        [0.5378, 0.6939, 0.4088],
        [0.9984, 0.7538, 0.0876],
        [0.7842, 0.9031, 0.8082],
        [0.2602, 0.5015, 0.0723],
        [0.8761, 0.8960, 0.1690],
        [0.6893, 0.0970, 0.2920],
        [0.5937, 0.2112, 0.5687],
        [0.9115, 0.8723, 0.1825],
        [0.1751, 0.8222, 0.9085],
        [0.3843, 0.4109, 0.0699],
        [0.1573, 0.0157, 0.7008]])

In [73]:
expanded_data

tensor([[[[0.9681, 0.1617, 0.0027],
          [0.9681, 0.1617, 0.0027],
          [0.9681, 0.1617, 0.0027]],

         [[0.9681, 0.1617, 0.0027],
          [0.9681, 0.1617, 0.0027],
          [0.9681, 0.1617, 0.0027]],

         [[0.9681, 0.1617, 0.0027],
          [0.9681, 0.1617, 0.0027],
          [0.9681, 0.1617, 0.0027]]],


        [[[0.0947, 0.6865, 0.0041],
          [0.0947, 0.6865, 0.0041],
          [0.0947, 0.6865, 0.0041]],

         [[0.0947, 0.6865, 0.0041],
          [0.0947, 0.6865, 0.0041],
          [0.0947, 0.6865, 0.0041]],

         [[0.0947, 0.6865, 0.0041],
          [0.0947, 0.6865, 0.0041],
          [0.0947, 0.6865, 0.0041]]],


        [[[0.1417, 0.8920, 0.0695],
          [0.1417, 0.8920, 0.0695],
          [0.1417, 0.8920, 0.0695]],

         [[0.1417, 0.8920, 0.0695],
          [0.1417, 0.8920, 0.0695],
          [0.1417, 0.8920, 0.0695]],

         [[0.1417, 0.8920, 0.0695],
          [0.1417, 0.8920, 0.0695],
          [0.1417, 0.8920, 0.0695]]],


    

In [28]:
np.prod((20, *observation_shape))

5400

In [29]:
np.prod(data.shape)

600

In [32]:
data.view(num_observations, *observation_shape).shape

RuntimeError: shape '[20, 10, 3, 3, 3]' is invalid for input of size 600

In [14]:
actor_output, critic_output = network(data)

RuntimeError: Expected 4D (unbatched) or 5D (batched) input to conv3d, but got input of size: [20, 10, 3]

In [12]:
actor_output.shape, critic_output.shape

(torch.Size([20, 1, 3, 1, 1]), torch.Size([20, 1]))

In [8]:
torch.Tensor.view_as??

Docstring:
view_as(other) -> Tensor

View this tensor as the same size as :attr:`other`.
``self.view_as(other)`` is equivalent to ``self.view(other.size())``.

Please see :meth:`~Tensor.view` for more information about ``view``.

Args:
    other (:class:`torch.Tensor`): The result tensor has the same size
        as :attr:`other`.
Type:      method_descriptor


In [9]:
data.shape

torch.Size([20, 10, 3, 3, 3])

In [7]:
# view tensor
data.view(20, )

Docstring:
view_as(other) -> Tensor

View this tensor as the same size as :attr:`other`.
``self.view_as(other)`` is equivalent to ``self.view(other.size())``.

Please see :meth:`~Tensor.view` for more information about ``view``.

Args:
    other (:class:`torch.Tensor`): The result tensor has the same size
        as :attr:`other`.
Type:      builtin_function_or_method


# 1x1 kernel (aka linear)

In [32]:

from open_spiel.python.pytorch.auction_convnet import AuctionConvNet, AuctionNet

In [33]:
observation_shape = (13, 7)
num_observations = 20
data = torch.rand(num_observations, *observation_shape)

network = AuctionNet(observation_shape, [50, 50], add_skip_connections=True, add_max_features=True)

In [34]:
actor_output, critic_output = network(data)

In [35]:
actor_output.shape

torch.Size([20, 13])

In [36]:
critic_output.shape

torch.Size([20, 1])

In [37]:
actor_output

tensor([[ 0.0034,  0.0049,  0.0041,  0.0017,  0.0035,  0.0041,  0.0049,  0.0032,
          0.0056,  0.0058,  0.0036,  0.0025,  0.0045],
        [ 0.0027,  0.0022,  0.0026,  0.0018,  0.0030,  0.0033,  0.0022,  0.0032,
          0.0020,  0.0023,  0.0027,  0.0019,  0.0026],
        [ 0.0008,  0.0016,  0.0029,  0.0004,  0.0008,  0.0020,  0.0011,  0.0009,
          0.0003,  0.0014,  0.0020,  0.0025,  0.0022],
        [ 0.0027,  0.0023,  0.0036,  0.0016,  0.0026,  0.0018,  0.0015,  0.0026,
          0.0029,  0.0030,  0.0031,  0.0038,  0.0030],
        [ 0.0046,  0.0034,  0.0028,  0.0052,  0.0032,  0.0017,  0.0037,  0.0031,
          0.0046,  0.0029,  0.0037,  0.0040,  0.0056],
        [ 0.0025,  0.0019,  0.0015,  0.0022,  0.0028,  0.0016,  0.0016,  0.0016,
          0.0014,  0.0007,  0.0013,  0.0015,  0.0011],
        [ 0.0040,  0.0031,  0.0030,  0.0041,  0.0032,  0.0034,  0.0043,  0.0047,
          0.0036,  0.0028,  0.0033,  0.0038,  0.0046],
        [ 0.0017,  0.0016,  0.0012,  0.0021,  0.

In [22]:
torch.cat([data, torch.max(data, dim=1, keepdim=True)[0].expand_as(data)], dim=2).shape

torch.Size([20, 13, 14])

In [14]:
torch.max(data, dim=1, keepdim=True)[0].expand_as(data).shape

torch.Size([20, 13, 7])